# 📊 Notebook 02: Przygotowanie danych

**Cel:** Załadowanie zebranych recenzji, podstawowa eksploracja danych oraz przygotowanie próbki do ćwiczeń klasyfikacji.

---

## Co robimy w tym notebooku?

1. Ładujemy recenzje ze Steam (wynik scrapera z Notebook 01)
2. Explorujemy dane - ile recenzji, jak wyglądają, jakie mają cechy
3. Przygotowujemy **próbkę roboczą** do ćwiczeń
4. Tworzymy **golden dataset** - ręcznie etykietowaną próbkę do ewaluacji

---

> 💡 **Golden dataset** to zbiór przykładów z poprawnymi odpowiedziami, który służy jako punkt odniesienia do oceny jakości modelu. W tym warsztacie stworzymy go ręcznie na podstawie próbki recenzji.

## ⚙️ Setup - środowisko

Ta komórka wykrywa czy jesteś w **Google Colab** czy lokalnie, a następnie:
1. Klonuje repozytorium z GitHub (tylko Colab)
2. Instaluje wymagane biblioteki
3. Konfiguruje ścieżki importów

In [ ]:
import sys
import subprocess
from pathlib import Path

# Wykryj środowisko
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"{'🟡 Google Colab' if IN_COLAB else '💻 Lokalne środowisko'}")

# ============================================================
# KONFIGURACJA REPO (zmień URL na właściwe przed warsztatami)
# ============================================================
REPO_URL = "https://github.com/JSerek/techland-genai-workshop.git"
REPO_DIR = Path("/content/szkolenie_techland") if IN_COLAB else Path(".").resolve().parent
# ============================================================

if IN_COLAB:
    if not REPO_DIR.exists():
        print(f"📥 Klonuję repo...")
        subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
        print("✅ Repo sklonowane")
    else:
        print("🔄 Aktualizuję repo (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)

    # Dodaj repo do sys.path
    sys.path.insert(0, str(REPO_DIR))

    # Zainstaluj zależności
    print("📦 Instaluję biblioteki...")
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", str(REPO_DIR / "requirements.txt"), "-q"],
        check=True
    )
    print("✅ Biblioteki zainstalowane")
else:
    # Lokalnie: dodaj root projektu do sys.path
    sys.path.insert(0, str(REPO_DIR))
    print(f"📂 Ścieżka projektu: {REPO_DIR}")

print("\n✅ Setup zakończony")

In [ ]:
# ============================================================
# 🔑 Konfiguracja API
# W Google Colab: dodaj sekrety w menu po lewej (🔑 ikona)
#   Nazwy sekretów: VERTEX_AI_API_KEY, VERTEX_AI_BASE_URL, MODEL_NAME
#
# Lokalnie: ustaw zmienne środowiskowe lub wpisz wartości bezpośrednio
# ============================================================

if IN_COLAB:
    from google.colab import userdata
    API_KEY   = userdata.get("VERTEX_AI_API_KEY")
    BASE_URL  = userdata.get("VERTEX_AI_BASE_URL")
    MODEL_NAME = userdata.get("MODEL_NAME") or "google/gemini-2.5-flash-lite"
else:
    import os
    API_KEY   = os.environ.get("VERTEX_AI_API_KEY", "TODO: wklej API key")
    BASE_URL  = os.environ.get("VERTEX_AI_BASE_URL", "TODO: wklej endpoint URL")
    MODEL_NAME = os.environ.get("MODEL_NAME", "google/gemini-2.5-flash-lite")

# Walidacja
if not API_KEY or API_KEY == "TODO: wklej API key":
    print("❌ Brak API_KEY! Dodaj sekret 'VERTEX_AI_API_KEY' w Colab Secrets.")
elif not BASE_URL or BASE_URL == "TODO: wklej endpoint URL":
    print("❌ Brak BASE_URL! Dodaj sekret 'VERTEX_AI_BASE_URL' w Colab Secrets.")
else:
    print(f"✅ API skonfigurowane: {MODEL_NAME}")
    print(f"   Endpoint: {BASE_URL[:60]}...")

## 1. Ładowanie danych

Dane zostały wcześniej zebrane za pomocą scrapera Steam (Notebook 01).
Ładujemy je z pliku CSV.

In [ ]:
# ============================================================
# KONFIGURACJA - zmień ścieżkę do pliku z danymi
# ============================================================
DATA_PATH = "../data/raw/steam_reviews.csv"  # TODO: zaktualizuj nazwę pliku
# ============================================================

data_file = Path(DATA_PATH)

if not data_file.exists():
    print(f"❌ Nie znaleziono pliku: {DATA_PATH}")
    print("Upewnij się że uruchomiłeś Notebook 01 i dane zostały zapisane.")
else:
    df = pd.read_csv(data_file)
    print(f"✅ Załadowano {len(df):,} recenzji")
    print(f"Kolumny: {list(df.columns)}")

## 2. Eksploracja danych

In [ ]:
# Podstawowe statystyki
print("=" * 50)
print("PODSTAWOWE STATYSTYKI")
print("=" * 50)
print(f"Liczba recenzji:          {len(df):,}")
print(f"Pozytywne:                {df['sentiment'].value_counts().get('positive', 0):,}")
print(f"Negatywne:                {df['sentiment'].value_counts().get('negative', 0):,}")
print(f"\nŚrednia długość recenzji: {df['review_text'].str.len().mean():.0f} znaków")
print(f"Mediana długości:         {df['review_text'].str.len().median():.0f} znaków")
print(f"Max długość:              {df['review_text'].str.len().max():,} znaków")

In [ ]:
# Wyświetl przykładowe recenzje
print("\nPRZYKŁADOWE RECENZJE:")
print("=" * 50)
for i, row in df.sample(5, random_state=42).iterrows():
    print(f"[{row['sentiment'].upper()}] {row['review_text'][:200]}...")
    print("-" * 40)

In [ ]:
# Rozkład długości recenzji
df["review_length"] = df["review_text"].str.len()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram długości
axes[0].hist(df["review_length"].clip(upper=2000), bins=40, color="#4C72B0", edgecolor="white")
axes[0].set_xlabel("Długość recenzji (znaki)")
axes[0].set_ylabel("Liczba recenzji")
axes[0].set_title("Rozkład długości recenzji")

# Długość per sentyment
df.boxplot(column="review_length", by="sentiment", ax=axes[1])
axes[1].set_title("Długość recenzji per sentyment")
axes[1].set_xlabel("Sentyment")
axes[1].set_ylabel("Długość (znaki)")
plt.suptitle("")

plt.tight_layout()
plt.show()

## 3. Przygotowanie próbki roboczej

Do ćwiczeń klasyfikacji potrzebujemy **zarządzalnej próbki** recenzji.
Na warsztacie będziemy używać ok. 50-100 recenzji.

In [ ]:
# ============================================================
# KONFIGURACJA próbki
# ============================================================
SAMPLE_SIZE = 50           # Liczba recenzji do ćwiczeń
RANDOM_SEED = 42           # Dla reprodukowalności
MIN_REVIEW_LENGTH = 50     # Minimalna długość recenzji (filtrowujemy zbyt krótkie)
MAX_REVIEW_LENGTH = 1000   # Maksymalna długość (opcjonalnie)
# ============================================================

# Filtrowanie
df_filtered = df[
    (df["review_length"] >= MIN_REVIEW_LENGTH) &
    (df["review_length"] <= MAX_REVIEW_LENGTH)
].copy()

print(f"Po filtrowaniu: {len(df_filtered):,} recenzji")

# Próbka
df_sample = df_filtered.sample(n=min(SAMPLE_SIZE, len(df_filtered)), random_state=RANDOM_SEED)
df_sample = df_sample.reset_index(drop=True)

print(f"Próbka robocza: {len(df_sample)} recenzji")
df_sample[["sentiment", "review_text"]].head(5)

In [ ]:
# Zapisz próbkę
sample_path = Path("../data/processed/workshop_sample.csv")
sample_path.parent.mkdir(parents=True, exist_ok=True)
df_sample.to_csv(sample_path, index=False)
print(f"✅ Próbka zapisana: {sample_path}")

## 4. Golden Dataset

> 🚧 **TODO po analizie danych**
>
> Ten sekcja zostanie uzupełniona po:
> 1. Określeniu kategorii tematycznych (analiza jakościowa recenzji)
> 2. Ręcznym etykietowaniu próbki recenzji
>
> Kategorie tematyczne będą wynikiem analizy — np. `bug`, `performance`, `story`, `gameplay`, `graphics` itd.

**Format golden dataset:**
```json
[
  {
    "review_text": "The game crashes every 20 minutes...",
    "labels": ["bug", "performance"]
  },
  {
    "review_text": "Amazing story but repetitive gameplay...",
    "labels": ["story", "gameplay"]
  }
]
```

In [ ]:
# ============================================================
# TODO: Uzupełnij po analizie danych
# ============================================================

# Ścieżka do golden dataset
GOLDEN_DATASET_PATH = "../data/evaluation/golden_dataset.json"  # TODO

# Kategorie tematyczne
CATEGORIES = [
    # TODO: uzupełnij po analizie danych
    # Przykłady: "bug", "performance", "story", "gameplay", "graphics", "price", "other"
]

print("⚠️  Golden dataset nie jest jeszcze zdefiniowany.")
print(f"Po uzupełnieniu wczytaj go z: {GOLDEN_DATASET_PATH}")

# Tak będzie wyglądało wczytanie golden dataset:
# golden_path = Path(GOLDEN_DATASET_PATH)
# if golden_path.exists():
#     with open(golden_path) as f:
#         golden_data = json.load(f)
#     golden_texts = [item["review_text"] for item in golden_data]
#     golden_labels = [item["labels"] for item in golden_data]
#     print(f"✅ Golden dataset: {len(golden_data)} przykładów")

---

## ✅ Podsumowanie

Po uruchomieniu tego notebooka masz:
- **`data/processed/workshop_sample.csv`** - próbka recenzji do ćwiczeń
- Podstawowe rozumienie struktury danych

**Następny krok:** Notebook 03 - Iteracja 1: Podstawowe promptowanie